In [2]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import gc
from tqdm import tqdm
from datetime import datetime, timedelta
import openpyxl
pd.options.display.max_columns = 100
pd.options.display.max_rows=1000

pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


In [3]:
access_key = pd.read_csv('../../cp_data.csv').iloc[0,1]
secret_key = pd.read_csv('../../cp_data.csv').iloc[1,1]

In [7]:
report_type = 'orders'
report_type = 'cancels'

In [225]:
# # 원본
# import requests 
# import hmac
# import hashlib
# from time import gmtime, strftime

# REQUEST_METHOD = "GET"
# DOMAIN = "https://api-gateway.coupang.com"
# URL = f"/v2/providers/affiliate_open_api/apis/openapi/v1/reports/{report_type}"

# # Replace with your own ACCESS_KEY, SECRET_KEY, startDate, endDate
# ACCESS_KEY = access_key
# SECRET_KEY = secret_key
# startDate = "20240403"
# endDate = "20240508"
# # Assuming subId is not required, if needed add it to the query parameters

# def generateHmac(method, url, secretKey, accessKey, queryString=""):
#     # Correct formatting for date-time if needed
#     datetimeGMT = strftime('%y%m%d', gmtime()) + 'T' + strftime('%H%M%S', gmtime()) + 'Z'
#     message = datetimeGMT + method + url + queryString

#     signature = hmac.new(bytes(secretKey, 'utf-8'), 
#                          message.encode('utf-8'), 
#                          hashlib.sha256).hexdigest()

#     return "CEA algorithm=HmacSHA256, access-key={}, signed-date={}, signature={}".format(accessKey, datetimeGMT, signature)


# def fetch_data(page):
#     queryString = "startDate={}&endDate={}&page={}".format(startDate, endDate, page)
#     fullURL = "{}{}?{}".format(DOMAIN, URL, queryString)
#     authorization = generateHmac(REQUEST_METHOD, URL, SECRET_KEY, ACCESS_KEY, queryString)
#     response = requests.request(method=REQUEST_METHOD, url=fullURL,
#                                 headers={
#                                     "Authorization": authorization,
#                                     "Content-Type": "application/json"
#                                 })
#     return response.json()

# def main():
#     page = 0
#     results = []

#     while True:
#         try:
#             response = fetch_data(page)
#             if 'data' in response:
#                 results.extend(response['data'])
#                 if len(response['data']) < 1000:
#                     break
#                 page += 1
#                 print(response['data'])
#             else:
#                 # Handle errors or missing 'data' key
#                 print(f"Error retrieving data: {response.get('message', 'No error message available')}")
#                 break
#         except:
#             print("pass")
#         print(f"Total items fetched: {len(results)}")

#     return results

# # Execute the main function to handle pagination
# all_data = main()
# df = pd.DataFrame(all_data)

Error retrieving data: No error message available


In [9]:
## 날짜 페이징
import requests 
import hmac
import hashlib
from time import gmtime, strftime
from datetime import datetime, timedelta
import pandas as pd

REQUEST_METHOD = "GET"
DOMAIN = "https://api-gateway.coupang.com"
URL = f"/v2/providers/affiliate_open_api/apis/openapi/v1/reports/{report_type}"


def generateHmac(method, url, secretKey, accessKey, queryString=""):
    datetimeGMT = strftime('%y%m%d', gmtime()) + 'T' + strftime('%H%M%S', gmtime()) + 'Z'
    message = datetimeGMT + method + url + queryString
    signature = hmac.new(bytes(secretKey, 'utf-8'), message.encode('utf-8'), hashlib.sha256).hexdigest()
    return f"CEA algorithm=HmacSHA256, access-key={accessKey}, signed-date={datetimeGMT}, signature={signature}"

from tqdm import tqdm  # Import tqdm

def fetch_data(startDate, endDate, page):
    queryString = f"startDate={startDate}&endDate={endDate}&page={page}"
    fullURL = f"{DOMAIN}{URL}?{queryString}"
    authorization = generateHmac(REQUEST_METHOD, URL, SECRET_KEY, ACCESS_KEY, queryString)
    headers = {"Authorization": authorization, "Content-Type": "application/json"}
    response = requests.get(fullURL, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to fetch data: HT|TP {response.status_code} - {response.text}")
        return None
    
    try:
        return response.json()
    except ValueError:
        print(f"Failed to decode JSON from response: {response.text}")
        return None

def fetch_data_in_chunks(start_date, end_date):
    start = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.strptime(end_date, '%Y%m%d')
    results = []

    date_range = (end - start).days // 30 + 1  # Calculate the number of chunks
    date_chunks = tqdm(range(date_range), desc="Fetching date ranges")

    for _ in date_chunks:
        chunk_end = start + timedelta(days=29)
        if chunk_end > end:
            chunk_end = end

        page = 0
        while True:
            response = fetch_data(start.strftime('%Y%m%d'), chunk_end.strftime('%Y%m%d'), page)
            if response:
                data = response.get('data', [])
                if data:
                    results.extend(data)
                    if len(data) < 1000:
                        break
                    page += 1
                else:
                    break
            else:
                break

        start = chunk_end + timedelta(days=1)
        if start > end:
            break

    return results

# Define your access keys and date range
ACCESS_KEY = access_key
SECRET_KEY = secret_key
start_date = "20240301"
end_date = "20240509"

# Collect all data
all_data = fetch_data_in_chunks(start_date, end_date)
df = pd.DataFrame(all_data)



Fetching date ranges:  33%|███▎      | 1/3 [00:03<00:07,  3.77s/it]

Failed to fetch data: HT|TP 504 - {
  "code" : "ERROR",
  "message" : "Request timed out, if the situation continues consider applying timeout extension.",
  "transactionId" : "f9aee26e-d51f-4f1f-b9ff-d3a2fdcb2ecc"
}


Fetching date ranges:  67%|██████▋   | 2/3 [00:06<00:03,  3.42s/it]

Failed to fetch data: HT|TP 504 - {
  "code" : "ERROR",
  "message" : "Request timed out, if the situation continues consider applying timeout extension.",
  "transactionId" : "d0a34e83-083a-43ae-9c7c-e5cfe9c3d372"
}


Fetching date ranges:  67%|██████▋   | 2/3 [00:07<00:03,  3.84s/it]


In [6]:
# df.to_parquet('../result/0301-0509_cp_orderReport_2.parquet')

df.to_parquet('../result/0301-0509_cp_cancelReport_2.parquet')

In [310]:
cancel = pd.read_parquet('../result/0301-0509_cp_cancelReport_2.parquet')
order = pd.read_parquet('../result/0301-0509_cp_orderReport_2.parquet')
origin = pd.read_excel('../result/129603-daily-trend-AF3155932-2024-04-03-2024-05-01.xlsx',engine='openpyxl',sheet_name=1)

order.rename(columns={'date':'orderDate'},inplace=True)

In [311]:
order.gmv.sum() + cancel.query("date >= '20240403'").gmv.sum()

1189406999.01

In [312]:
lst = cancel.columns.tolist()
del lst[1]
lst[0:8]

['orderDate',
 'trackingCode',
 'subId',
 'subParam',
 'addtag',
 'ctag',
 'orderId',
 'productId']

2 행 정도 빔. 당연한 오차

In [313]:
tt = pd.merge(cancel.query("orderDate >= '20240403' and orderDate <= '20240501' ") ,order, on=lst[0:8] ,how='left',indicator=True)
tt[tt['_merge']=='left_only']

,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName_x,quantity_x,gmv_x,commissionRate_x,commission_x,productName_y,quantity_y,gmv_y,commissionRate_y,commission_y,_merge
68,20240410,20240502,AF3155932,homecoin,8f8ac1bc-37a0-4640-a921-fb15b80e28fb,400,6382265775,8244947719083851,7873223097,카멜레온360 송풍구형 스마트폰 차량용 거치대 A type,-1,-18900.0,5.25,-993.0,NaN,NaN,NaN,NaN,NaN,left_only
2236,20240411,20240412,AF3155932,homecoin,8b35cd09-b081-41bb-8b8f-1ab6e56f3422,460,6927276760,8004943516605005,8026199285,다누리 뽀로로 복숭아맛 유아 비타민C 60g,-1,-3720.0,5.25,-196.0,NaN,NaN,NaN,NaN,NaN,left_only
2408,20240411,20240412,AF3155932,homecoin,00558564-50a3-4eb6-aaa2-18cc2ea50b27,460,7320739567,8864943316144724,8026199285,다누리 뽀로로 복숭아맛 유아 비타민C 60g,-3,-11160.0,5.25,-586.0,NaN,NaN,NaN,NaN,NaN,left_only


데이터 처리

In [314]:
lst = order.columns.tolist()

lst
len(pd.merge(order, cancel.query("orderDate >= '20240403' and orderDate <= '20240501' "), on=lst[0:8] ,how='left',indicator=True)),len(order)

(80224, 80218)

In [315]:
# 중복 키 확인
duplicate_keys = cancel.duplicated(subset=lst[0:8], keep=False)
if duplicate_keys.any():
    print(f"Duplicate keys found in 'cancel': {cancel[duplicate_keys]}")
    # 중복 제거 또는 다른 방식의 처리를 결정할 수 있음
    # 예: 중복 제거
    cancel = cancel.drop_duplicates(subset=lst[0:8], keep='first')


Duplicate keys found in 'cancel':       orderDate      date trackingCode     subId  \
index                                              
204    20240319  20240503    AF3155932  homecoin   
413    20240413  20240504    AF3155932  homecoin   
615    20240406  20240506    AF3155932  homecoin   
114    20240301  20240301    AF3155932  homecoin   
1042   20240304  20240306    AF3155932  homecoin   
1203   20240306  20240307    AF3155932  homecoin   
1395   20240306  20240308    AF3155932  homecoin   
1813   20240304  20240310    AF3155932  homecoin   
2034   20240301  20240312    AF3155932  homecoin   
3278   20240310  20240318    AF3155932  homecoin   
3374   20240316  20240318    AF3155932  homecoin   
3645   20240319  20240320    AF3155932  homecoin   
3671   20240316  20240320    AF3155932  homecoin   
5054   20240310  20240327    AF3155932  homecoin   
5792   20240330  20240330    AF3155932  homecoin   
5860   20240330  20240331    AF3155932  homecoin   
7470   20240405  20240406    A

In [316]:
# 병합 실행
merged_df = pd.merge(order, cancel.query("orderDate >= '20240403' and orderDate <= '20240501'"), on=lst[0:8], how='left', indicator=True)

# 병합 결과에서 'left_only'는 'order' 데이터프레임에만 있는 데이터
# 'both'는 두 데이터프레임에 모두 존재하는 데이터
only_in_order = merged_df[merged_df['_merge'] == 'left_only']
both_in = merged_df[merged_df['_merge'] == 'both']

print(f"Original Order Length: {len(order)}")
print(f"Merged DataFrame Length: {len(merged_df)}")
print(f"Only in Order DataFrame Length: {len(only_in_order)}")
print(f"Both in DataFrames Length: {len(both_in)}")

# 중복 제거 후 데이터 확인
merged_df.drop_duplicates(subset=lst[0:8], keep='first', inplace=True)
print(f"Length after removing duplicates: {len(merged_df)}")

merged_df = merged_df.iloc[:,0:-1]
merged_df.rename(columns={'productName_x':'productName','quantity_x':'quantity','commissionRate_x':'commissionRate','commission_x':'commission','gmv_x':'gmv',\
                          'quantity_y':'refund_quantity','commission_y':'refund_commission','gmv_y':'refund_gmv'
                          },inplace=True)
merged_df.drop(['commissionRate_y','productName_y','commissionRate_y'],axis=1,inplace=True)
merged_df.fillna(0,inplace=True)

merged_df['total_gmv'] = merged_df['gmv'] + merged_df['refund_gmv']
merged_df['total_commission'] = merged_df['commission'] + merged_df['refund_commission']
merged_df['total_quantity'] = merged_df['quantity'] + merged_df['refund_quantity']

Original Order Length: 80218
Merged DataFrame Length: 80218
Only in Order DataFrame Length: 74526
Both in DataFrames Length: 5692
Length after removing duplicates: 80218


검증
오더날짜 기준 (오더 건과 해당 취소건만 집계)
일일 날짜 기준 (해당 날짜의 오더건과 해당일 발생한 이전 주문 총합의 취소건)

일일 날짜 기준으로 취소건을 집계할 경우, 오더날짜 기준의 취소건 대비 많으므로 조금 더 많은 주문 수와 매출이 집계됨

매출로 1천만원, 

In [317]:
## 오더 기준 주문 정보 (API)
print(merged_df.total_quantity.sum())
print(merged_df.total_gmv.sum())
print(merged_df.total_commission.sum())

96845.0
1220113724.01
61258829.0


In [318]:
## 오더 기준 주문 정보 (API) + 무정보 데이터 제외
print(merged_df.query("subId != '' and total_gmv > 0 ").total_quantity.sum())
print(merged_df.query("subId != '' and total_gmv > 0 ").total_gmv.sum())
print(merged_df.query("subId != '' and total_gmv > 0 ").total_commission.sum())

96654.0
1219449378.01
61226111.0


In [319]:
## 일일 기준 주문 정보 (API)
print(order.quantity.sum() + cancel.query("date >= '20240403' and date <= '20240501'").quantity.sum())
print(order.gmv.sum() + cancel.query("date >= '20240403' and date <= '20240501'").gmv.sum())
print(order.commission.sum() + cancel.query("date >= '20240403' and date <= '20240501'").commission.sum())

96727
1216498106.01
61064482.0


In [320]:
order.columns

Index(['orderDate', 'trackingCode', 'subId', 'subParam', 'addtag', 'ctag',
       'orderId', 'productId', 'productName', 'quantity', 'gmv',
       'commissionRate', 'commission'],
      dtype='object')

In [321]:
## 일일 기준 주문 정보 (리포트)
origin['total_quantity'] = origin['판매 수량'] + origin['취소 수량']
origin['total_gmv'] = origin['매출 액'] + origin['취소 액']
origin['total_commission'] = origin['수익'] + origin['취소 수익']

print(origin.total_quantity.sum())
print(origin.total_gmv.sum())
print(origin.total_commission.sum())
print(len(origin))

81966
1020047124
52061504
62559


In [322]:
merged_df.total_gmv.sum()

1220113724.01

In [327]:
merged_df.to_excel('../result/result_0403_0509.xlsx',engine='openpyxl')
merged_df.to_csv('../result/result_0403_0509.csv',encoding='utf-8-sig')